# Notebook de Avaliação Quantitativa

- Objetivo: Responsável por carregar os modelos, criar a pipeline completa, executar a pipeline e realizar a avaliação quantitativa nos dados e validação e teste

In [ ]:
## Descomente os comandos bash abaixo para verificar informações pertinenetes sobre o hardware (GPU e CPU) do sistema (assume sistema Linux)
# !grep 'model name' /proc/cpuinfo | uniq
# !nvidia-smi

model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
Fri Sep 26 16:53:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---

In [ ]:
!git clone  https://github.com/marceloluisr/Desafio_INOVAI_2025.git

Cloning into 'deteccao_pontos_chaves'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 127 (delta 1), reused 4 (delta 1), pack-reused 121 (from 1)
Receiving objects: 100% (127/127), 33.00 MiB | 27.50 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [ ]:
USE_COLAB = True  # ou Falso dependendo do ambiente.

Vamos montar o drive no ambiente de execução para acessar o dados. IMPORTANTE:
a montagem do drive requer permitir acesso do Google Colab para o drive e os dados devem estar disponíveis.


In [ ]:
if USE_COLAB:
  # montando google drive no ambiente para acessar dataset
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

codigo_base_path = "/content/Desafio_INOVAI_2025/src" # [ALTERE AQUI COM O PATH DO CÓDIGO CLONADO]


# isso permite o import do módulos do código fonte no jupyter notebook
sys.path.append(codigo_base_path)
# para remover descomente  o bloco de código linha abaixo
# if codigo_base_path in sys.path:
#     sys.path.remove(codigo_base_path)

IMPORTANTE APENAS SE VOCÊ NÃO ESTIVER NO GOOGLE COLAB: antes de prosseguir, recomendo executar esse notebook dentro de um ambiente conda ou pyenv.

In [ ]:
path_requisitos = "{}/requirements.txt".format(codigo_base_path)
if not USE_COLAB:
    %pip install -r {path_requisitos}
else:
    %pip install xgboost==3.0.5 mediapipe==0.10.21 rt_pose==0.0.3


AQUI2
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/lib/python3.12/importlib/metadata/__init__.py", line 819, in read_text
    return self._path.joinpath(filename).read_text(encoding='utf-8')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/pathlib.py", line 1027, in read_text
    with self.open(mode='r', encoding=encoding, errors=errors) as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/pathlib.py", line 1013, in open
    return io.open(self, mode, buffering, encoding, errors, newline)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.12/dist-packages/nvidia_cufile_cu12-1.11.1.6.dist-info/requires.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_

In [ ]:
# !pip install \
#   xgboost==3.0.5 \
#   mediapipe==0.10.21 \
#   rt_pose==0.0.3


# Imports

In [ ]:
from utils.utils import carregar_lista_de_txt, carregar_dataframe, unnormalize_tensor, salvar_silhuetas
from models.feature_extractor import ExtractFeaturesKeypoints
from models.kp_modules import KeypointsModule, MediaPipeKeypointsModule, RTPoseKeypointDetectorModule
from models.reg_modules import RegressionModule, XGBoostRegressionModule, DepthwiseSeparableConv1DRegressor, DepthwiseSeparableConv1DRegressorModule
from models.seg_modules import  DeepLabSegmentationModule, U2NetSegmentationModule
from dataset.dataset import RegressaoDataset
import torch
import yaml
import os
import math
import zipfile
import joblib
from torch.utils.data import DataLoader, Dataset
import random
import numpy as np
import subprocess
from PIL import Image

from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
  result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
  print(result.stdout)

Sat Sep 27 06:21:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
## Carrega o arquivo de configuração para reproduzir os experimentos

file_path = os.path.join(codigo_base_path, "configuration.yaml")

with open(file_path, "r") as file:
    config = yaml.safe_load(file)

print(config)


{'colab': True, 'dataset_path': '/content/drive/MyDrive/Colab Notebooks/SelecaoINOVIA/datasets', 'destination_path': '/content/unzipped_files', 'log_path': '/content/src/log', 'regressor_module_name': 'XGBoost', 'keypoint_module_name': 'RTPose', 'segmentation_module_name': 'U2Net', 'train': {'batch_size': 120, 'learning_rate': '1e-7', 'u2net_seg_human_model_path': ''}}


In [ ]:
def set_torch_seed(seed: int = 42):
    """
    Define uma semente global para garantir reprodutibilidade em PyTorch, NumPy e random.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # para múltiplas GPUs

    # Garante comportamento determinístico (pode impactar performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    print(f"Semente definida: {seed}")
set_torch_seed(2025)

Semente definida: 2025


In [ ]:
diretorio_pasta_dados = config['dataset_path']

# List files
os.listdir(diretorio_pasta_dados)


['homens_15k.zip', 'medidas_dados_sinteticos.csv', 'mulheres_15k.zip']

In [ ]:

# Pasta de destino para salvar as imagens retirada dos arquivos compcatos
# no colab [ALTERE AQUI se necessário], e será removido se desligar o ambiente.
destination_path = config['destination_path']


In [ ]:
## Unzip as pastas compactadas  'homens_15k.zip' e 'mulheres_15k.zip' contidas em diretorio_pasta_dados
#   para destination_path
if USE_COLAB:
  zip_file_paths = [
      diretorio_pasta_dados + '/mulheres_15k.zip',
      diretorio_pasta_dados + '/homens_15k.zip'
  ]

  os.makedirs(destination_path, exist_ok=True)

  # Unzip
  for zip_file_path in zip_file_paths:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_path)

## Criando Dataset Pytorch

In [ ]:
config['log_path'] = os.path.join(codigo_base_path, "log")

In [ ]:
treino_diretorios_imagens = carregar_lista_de_txt(os.path.join(config['log_path'], "treino_diretorios_imagens.txt") )
df_medidas_filtrado_treino = carregar_dataframe(os.path.join(config['log_path'], "df_medidas_filtrado_treino.csv") )

validacao_diretorios_imagens = carregar_lista_de_txt(os.path.join(config['log_path'], "validacao_diretorios_imagens.txt") )
df_medidas_filtrado_val = carregar_dataframe(os.path.join(config['log_path'], "df_medidas_filtrado_val.csv") )


teste_diretorios_imagens = carregar_lista_de_txt(os.path.join(config['log_path'], "teste_diretorios_imagens.txt") )
df_medidas_filtrado_teste = carregar_dataframe(os.path.join(config['log_path'], "df_medidas_filtrado_teste.csv") )

In [ ]:

print("Total de imagens para treino: ", len(treino_diretorios_imagens))
print("Total de imagens para validação: ", len(validacao_diretorios_imagens))
print("Total de imagens para teste: ", len(teste_diretorios_imagens))

Total de imagens para treino:  15392
Total de imagens para validação:  4948
Total de imagens para teste:  9272


In [ ]:
config['log_path']  =  os.path.join(codigo_base_path, "log")
if config['segmentation_module_name'] == 'DeepLabV3':
  seg_module = DeepLabSegmentationModule(device=device)
elif config['segmentation_module_name'] == 'U2Net':
  seg_module = U2NetSegmentationModule(
      config['train']['u2net_seg_human_model_path'],
      device
  )
else:
   raise ValueError(f"Módulo de segmentação desconhecido: {config['segmentation_module_name']}")



In [ ]:
config['log_path']  =  os.path.join(codigo_base_path, "log")
if config['keypoint_module_name'] == 'RTPose':
  kp_module = RTPoseKeypointDetectorModule()
elif config['keypoint_module_name'] == 'BlazePose':
  kp_module = MediaPipeKeypointsModule()
elif config['keypoint_module_name'] == 'Heuristic':
  kp_module = KeypointsModule()
else:
   raise ValueError(f"Módulo de detecção de pontos chaves desconhecido: {config['keypoint_module_name']}")



2025-09-27 12:08:48.397 | INFO     | rt_pose.pipeline:_load_detector:70 - Loading detector from `PekingU/rtdetr_r50vd_coco_o365`...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-27 12:08:51.239 | INFO     | rt_pose.pipeline:_load_detector:74 - Detector loaded to `cuda` with dtype `torch.float32`!
2025-09-27 12:08:51.240 | INFO     | rt_pose.pipeline:_load_pose_estimator:79 - Loading pose estimator from `usyd-community/vitpose-plus-small`...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-27 12:08:53.484 | INFO     | rt_pose.pipeline:_load_pose_estimator:83 - Pose estimator loaded to `cuda` with dtype `torch.float32`!
2025-09-27 12:08:53.484 | INFO     | rt_pose.pipeline:__init__:66 - Pipeline initialized successfully!


In [ ]:
config['log_path'] = os.path.join(codigo_base_path, "log")

if config['regressor_module_name'] == 'XGBoost':

  xgboost_model_dir = os.path.join(config['log_path'],"{}_{}_multioutput_xgb_model.pkl".format(
                   config['segmentation_module_name'],
                   config['keypoint_module_name']
  ))

  reg_module = XGBoostRegressionModule(
      model_dir = os.path.join(config['log_path'], xgboost_model_dir),
      scaler_dir = os.path.join(config['log_path'], 'treino_scaler.pkl'),
  )

elif config['regressor_module_name'] == 'DepthwiseSeparableConv1D':

  dsc_model_dir = os.path.join(
               config['log_path'],
               "{}_{}_separable_conv_regressor_weights.pt".format(
                   config['segmentation_module_name'],
                   config['keypoint_module_name']
                )
  )

  if isinstance(kp_module, MediaPipeKeypointsModule) or  isinstance(kp_module, RTPoseKeypointDetectorModule):
    reg_module =  DepthwiseSeparableConv1DRegressorModule(
      model_dir =  os.path.join(config['log_path'], dsc_model_dir),
      scaler_dir =  os.path.join(config['log_path'], 'treino_scaler.pkl'),
      device = device,
      input_length=55
    )
  else:
    reg_module =  DepthwiseSeparableConv1DRegressorModule(
      model_dir =  os.path.join(config['log_path'], dsc_model_dir),
      scaler_dir =  os.path.join(config['log_path'], 'treino_scaler.pkl'),
      device = device,
      input_length=21
    )
elif config['regressor_module_name'] == 'Heuristic':
  reg_module = RegressionModule()

else:
   raise ValueError(f"Módulo de detecção de regressão desconhecido: {config['regressor_module_name']}")


In [ ]:

feature_extractor = ExtractFeaturesKeypoints().to(device)


In [ ]:

treino_scaler = joblib.load(os.path.join(config['log_path'], "treino_scaler.pkl"))



val_dataset = RegressaoDataset(
    image_paths = validacao_diretorios_imagens,
    scaler = None,
    dataframe = df_medidas_filtrado_val,
    dataset_path = config['destination_path'],
    u2net = True if config['segmentation_module_name'] == 'U2Net' else False,
)



teste_dataset = RegressaoDataset(
    image_paths = teste_diretorios_imagens,
    scaler = None,
    dataframe = df_medidas_filtrado_teste,
    dataset_path = config['destination_path'],
    u2net = True if config['segmentation_module_name'] == 'U2Net' else False,
)



# Avaliação


Vamos usar a métrica de regressão Raiz do Erro Quadrático Médio, de Root Mean Squared Error (RMSE), que calcula
a média da norma quadrática entre os valores previstos e valores verdadeiros. Dentro do problema de Regressão Multivariada, comparamos vetores previstos e verdadeira para cada amostra de validação ou amostra de teste de modo que o resultado final é a média entre o erro quadrático de todas as amostras.

In [ ]:
# Criação do DataLoader para o split de validacão
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=1,
    shuffle=False,
    #pin_memory=True,
    #num_workers=2
)



# Criação do DataLoader para o split de teste
test_loader = DataLoader(
    dataset=teste_dataset,
    batch_size=1,
    shuffle=False,
    #pin_memory=True,
    #num_workers=2
)


In [ ]:
@torch.no_grad()
def print_average_rmse_gpu(loader, seg_module, kp_module, reg_module, device, feature_extractor_module=None, salvar_mascaras=False):
    global aux
    # O que é RMSE: https://c3.ai/glossary/data-science/root-mean-square-error-rmse/
    """
    Calcula e imprime o erro quadrático médio (RMSE) entre os valores reais e os valores previstos
    de medidas corporais em um problema de regressão multivariada, utilizando PyTorch.

    A função percorre um DataLoader contendo imagens e seus respectivos rótulos (medidas reais),
    extrai a silhueta da imagem, gera os keypoints, prediz as medidas com base nesses pontos,
    e calcula o RMSE considerando o vetor de medidas como uma única predição multivariada por amostra.

    Args:
    ----------
    loader : torch.utils.data.DataLoader
        DataLoader contendo tuplas (imagem, medidas reais).

    seg_module : módulo de segmentação
        Módulo responsável por extrair a silhueta da imagem.

    kp_module : módulo de keypoints
        Módulo responsável por extrair os pontos-chave da silhueta.

    reg_module : módulo de regressão
        Módulo responsável por prever as medidas corporais a partir dos keypoints.

    feature_extractor_module: módulo de extração de carcaterísticas dos pontos-chave
        Módulo responsável por extrair carcaterísticas dos pontos-chave da silhueta.

    device : torch.device
        Dispositivo onde os tensores serão processados (CPU ou GPU).

    Output:
    ---------
    - Imprime o valor médio de RMSE em pixels.
    """

    total_erro_quadratico = 0.0
    total_amostras = 0
    count = 0 # apenas uma variável acumuladora para salvar máscaras em sequência
    total_keypoints_not_detected = 0
    total_masks_not_segmented = 0

    for img, label in tqdm(loader):
        img, label = img.to(device), label.to(device) # move a imagem e os rótulos para o dispositivo.

        mask = seg_module.extract_silhouette(img) # extrai a silhueta da imagem.
        #seg_module.visualizar_silhueta(img, mask)
        if mask.max() > 0:
          if salvar_mascaras:
            salvar_silhuetas(img, mask, "./resultado", "amostra_{}".format(count))

        # se a silhueta estiver vazia, usa medidas zeradas
        if mask.max() == 0:
            total_masks_not_segmented = total_masks_not_segmented + 1

            measures = {
                'peito': 0.00, 'cintura': 0.00, 'quadril': 0.00, 'coxa': 0.00,
                'joelho': 0.00, 'panturrilha': 0.00, 'abdomen': 0.00,
                'pescoço': 0.00, 'biceps': 0.00
            }
        else:
          # gera os keypoints da silhueta.
          if isinstance(kp_module, MediaPipeKeypointsModule) or isinstance(kp_module, RTPoseKeypointDetectorModule):
              if isinstance(seg_module, U2NetSegmentationModule):
                img_unormalized = unnormalize_tensor(img.squeeze().cpu().detach())
                img_unormalized = (img_unormalized * 255).clip(0, 255).astype(np.uint8) # converte para RGB
              elif isinstance(seg_module, DeepLabSegmentationModule):
                img_unormalized = (img * 255).clamp(0, 255).byte() # converte para RGB
                img_unormalized = img_unormalized.squeeze().permute((1,2,0)).cpu().detach()
                img_unormalized = img_unormalized.numpy().astype(np.uint8)
              aux = img_unormalized.copy()
              keypoints = kp_module.detector.detect(
                  img_unormalized,
                  mask
              )

          else:
              keypoints = kp_module.extract_keypoints(mask)

          if keypoints is not None and feature_extractor is not None: # se  detectou pontos chaves
              keypoints_features = feature_extractor(keypoints).to(device) # 1xN (N=21 para KeypointDetectionModule e N=55 para MediaPipeKeypointsModule ou RTPoseKeypointDetectorModule)


              # prediz as medidas corporais com base nos keypoints.
              if isinstance(reg_module, XGBoostRegressionModule):
                measures = reg_module.predict_measures(np.expand_dims(keypoints_features.cpu().detach().numpy(), 0))
              elif isinstance(reg_module, DepthwiseSeparableConv1DRegressorModule):
                measures = reg_module.predict_measures(np.expand_dims(keypoints_features.cpu().detach().numpy(), 0))
              else:
                measures = reg_module.predict_measures(keypoints)

          else:
              # apenas acumula o total de falhas
              total_keypoints_not_detected =  total_keypoints_not_detected + 1

              measures = {
                  'peito': 0.00, 'cintura': 0.00, 'quadril': 0.00, 'coxa': 0.00,
                  'joelho': 0.00, 'panturrilha': 0.00, 'abdomen': 0.00,
                  'pescoço': 0.00, 'biceps': 0.00
              }
              #continue

          # # se a silhueta estiver vazia, usa medidas zeradas
          # if mask.max() == 0:
          #     total_masks_not_segmented = total_masks_not_segmented + 1

          #     measures = {
          #         'peito': 0.00, 'cintura': 0.00, 'quadril': 0.00, 'coxa': 0.00,
          #         'joelho': 0.00, 'panturrilha': 0.00, 'abdomen': 0.00,
          #         'pescoço': 0.00, 'biceps': 0.00
          #     }

        pred_tensor = torch.tensor(list(measures.values()), device=device)
        pred_tensor = pred_tensor.unsqueeze(0) # 1x9
        true_tensor = torch.tensor(label.cpu().detach().numpy(), device=device)
        true_tensor = true_tensor.view(1, -1) # 1x9
        assert pred_tensor.shape == true_tensor.shape

        # calculando o erro quadrático da predição (norma ao quadrado da diferença)
        erro_quadratico =  (pred_tensor - true_tensor).pow(2).sum().item()
        # acumulando o erro total e o número de amostras.
        total_erro_quadratico += erro_quadratico
        total_amostras += 1 # N

    # Computando RMSE de MSE
    erro_quadratico_medio = total_erro_quadratico / max(total_amostras, 1)
    raiz_erro_quadratico_medio = math.sqrt(erro_quadratico_medio)

    print(f"\n Média geral de RMSE: {raiz_erro_quadratico_medio:.2f} px")
    print("[INFO]Total de imagens com Pontos Chaves não detectados: ", total_keypoints_not_detected)
    print("[INFO] Total de imagens com máscaras previstas vazias: ", total_masks_not_segmented)


In [ ]:
config

{'colab': True,
 'dataset_path': '/content/drive/MyDrive/Colab Notebooks/SelecaoINOVIA/datasets',
 'destination_path': '/content/unzipped_files',
 'log_path': '/content/deteccao_pontos_chaves/log',
 'regressor_module_name': 'Heuristic',
 'keypoint_module_name': 'Heuristic',
 'segmentation_module_name': 'DeepLabV3',
 'train': {'batch_size': 120,
  'learning_rate': '1e-7',
  'u2net_seg_human_model_path': '/content/u2net_human_seg.pth'}}

In [ ]:
print("Pipeline = ", seg_module, "->", kp_module, "->", reg_module)

Pipeline =  <models.seg_modules.U2NetSegmentationModule object at 0x7ab7a1d9bf50> -> <models.kp_modules.RTPoseKeypointDetectorModule object at 0x7ab79df4bd40> -> <models.reg_modules.DepthwiseSeparableConv1DRegressorModule object at 0x7ab79c5920f0>


In [ ]:


import time

start_time = time.time()

# computação do RMSE para o conjunto de validação
print_average_rmse_gpu(
    loader = val_loader,
    seg_module = seg_module,
    kp_module = kp_module,
    reg_module = reg_module,
    feature_extractor_module = feature_extractor,
    device = device,
    salvar_mascaras=False
)

end_time = time.time()

elapsed_time = end_time - start_time

print(f"Tempo de execução: {elapsed_time:.2f} segundos")


100%|██████████| 4948/4948 [17:05<00:00,  4.82it/s]


 Média geral de RMSE: 25.78 px
[INFO]Total de imagens com Pontos Chaves não detectados:  4
[INFO] Total de imagens com máscaras previstas vazias:  0
Tempo de execução: 1025.69 segundos


In [ ]:

import time

start_time = time.time()


# computação do RMSE para o conjunto de teste
print_average_rmse_gpu(
    loader = test_loader,
    seg_module = seg_module,
    kp_module = kp_module,
    reg_module = reg_module,
    feature_extractor_module = feature_extractor,
    device = device,
    salvar_mascaras=False
)


end_time = time.time()

elapsed_time = end_time - start_time

print(f"Tempo de execução: {elapsed_time:.2f} segundos")

100%|██████████| 9272/9272 [21:36<00:00,  7.15it/s]


 Média geral de RMSE: 158.66 px
[INFO]Total de imagens com Pontos Chaves não detectados:  0
[INFO] Total de imagens com máscaras previstas vazias:  623
Tempo de execução: 1296.83 segundos


- Total de máscaras perdidas por DeepLabV3 na validação é 329 e no teste é 623.
- Total de máscaras perdidas por U2Net na validação é 0 e no teste é 0.


> **HARDWARE:** CPU Intel(R) Xeon(R) CPU @ 2.20GHz e GPU Nvidia Tesla T4.

| Segmentação | Ponto-chave | Regressor         | Tempo de inferência (s) | RMSE (px) validação | RMSE (px) teste | Total pontos chaves perdidos validação | Total pontos chaves perdidos teste |
|-------------|-------------|-------------------|--------------------------|----------------------|-----------------|----------------------------------------|-------------------------------------|
| DeepLabV3   | Heuristic   | Heuristic         | 21m33s                   | 160.76 px            | 158.66 px       | 0                                      | 0                                   |
| DeepLabV3   | RTPose      | XGBoost           | 35m46s                   | 55.80 px             | 55.80 px        | 2                                      | 3                                   |
| DeepLabV3   | RTPose      | DepthwiseConv1D   | 33m19s                   | 58.38 px             | 57.67 px        | 2                                      | 3                                   |
| DeepLabV3   | BlazePose   | DepthwiseConv1D   |            28m48s              |        109.41 px              |           110.62 px      |                    923                    |                    1872                 |
| DeepLabV3   | BlazePose   | XGBoost           |     30m42s                     |       108.65 px               |      109.95 px           |                923                        |               1872                      |
| DeepLabV3   | Heuristic   | DepthwiseConv1D   |       21m47s                   |       58.41 px                |     57.50 px            |               0                         |                 0                    |
| DeepLabV3   | Heuristic   | XGBoost           |      24m11s                    |    57.94 px                  |         57.35 px        |       0                                 |            0                         |
| U2Net       | RTPose      | DepthwiseConv1D   |             32m31s             |       25.78 px               |        25.11 px         |         4                               |                  7                   |
| U2Net       | RTPose      | XGBoost           |         34m52s                 |     **19.24 px**                 |         **19.18 px**        |       4                                 |             7                        |
| U2Net       | BlazePose   | DepthwiseConv1D   |     26m52s                     |        95.45 px              |         96.62 px        |                        921                |       1845                              |
| U2Net       | BlazePose   | XGBoost           |        29m17s                  |    94.42 px                  |          95.65 px       |                     921                   |                1845                     |
| U2Net       | Heuristic   | DepthwiseConv1D   |                 **18m49s**         |          25.28 px            |        24.80 px         |                  0                      |              0                       |
| U2Net       | Heuristic   | XGBoost           |    21m28s                      |        23.96 px              |     23.64 px            |               0                         |            0                         |



# Trabalhos futuros

- Conversão dos modelos para formatos apropriado para dispositivos móveis. Por exemplo, o trabalho de [Removebg](https://github.com/AppcentMobile/removebg) realizou a integração da U2Net para um apĺicativo android convertendo o modelo para formato .ptl (pytorch lite).
- Implantação dos melhores modelos para dentro de dispositivos móveis a fim de gerar resultados experimentais mais rigorosos que consideram principalmente o tempo de inferência (milisegundos) como métrica de latência.
- Avaliar mais profundamente a não detecção de pontos chaves em inúmeras imagens. A causa disso poderia ser relacionada aos próprios dados sintéticos?
